robik79@gmail.com <br>
https://github.com/rwlupus/master/blob/master/python/jupyter/rename_images_using_EXIF_DateTimeOriginal.ipynb  <br>
2024-02-12

In [ ]:
# Configuration

source_folder = 'path/to/source/folder'   # Update to your path
destination_folder = 'path/to/destination/folder' # Destination folder for copied files

conflict_resolution_method = 'increment_seconds'  # 'increment_seconds' or 'add_counter'
is_dry_run = True  # Set to False to make actual changes
rename_in_place = False  # Set to True to rename in place


 # imports
import os
import shutil
from datetime import datetime, timedelta
from PIL import Image
import pandas as pd
import openpyxl
from openpyxl.styles import PatternFill, Font
from openpyxl.utils import get_column_letter

# Helper functions
def read_exif_data(file_path):
    """Read and return EXIF DateTimeOriginal from an image file."""
    try:
        img = Image.open(file_path)
        exif_data = img._getexif()
        original_date = datetime.strptime(exif_data[36867], '%Y:%m:%d %H:%M:%S')
        return original_date
    except Exception as e:
        return None

def get_new_filename(original_date, conflict_count, method, original_extension):
    """Generate a new filename based on the original date, conflict count, method, and extension."""
    if method == 'increment_seconds':
        new_date = original_date + timedelta(seconds=conflict_count)
        return f"{new_date.strftime('%Y.%m.%d - %H.%M.%S')}{original_extension}"
    else:  # 'add_counter'
        return f"{original_date.strftime('%Y.%m.%d - %H.%M.%S')}_{conflict_count}{original_extension}"

def save_log_to_excel(log_entries, source_folder, conflict_resolution_method):
    """Save log entries to an Excel file, with formatting."""
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    log_filename = f"rename_log_{conflict_resolution_method}_{timestamp}.xlsx"
    log_path = os.path.join(os.path.dirname(source_folder), log_filename)
    df_log = pd.DataFrame(log_entries, columns=['Original Name', 'New Name', 'Error', 'Conflict'])
    
    # Save log using pandas
    df_log.to_excel(log_path, index=False, sheet_name="Log")

    # Open workbook and sheet for formatting
    wb = openpyxl.load_workbook(log_path)
    ws = wb.active

    # Set autofilter on all columns
    ws.auto_filter.ref = ws.dimensions

    # Autofit column widths and apply conditional formatting
    for col in ws.columns:
        max_length = max(len(str(cell.value)) for cell in col)
        ws.column_dimensions[get_column_letter(col[0].column)].width = max_length + 5

    # Apply conditional formatting
    for row in ws.iter_rows(min_row=2, max_row=ws.max_row, values_only=False):
        if row[2].value == 'TRUE':  # Error, red fill
            for cell in row:
                cell.fill = PatternFill(start_color="FF0000", end_color="FF0000", fill_type="solid")
        elif row[3].value == 'TRUE':  # Conflict, yellow fill
            for cell in row:
                cell.fill = PatternFill(start_color="FFFF00", end_color="FFFF00", fill_type="solid")

    wb.save(log_path)
    return log_path

def process_images_in_folder(source_folder, destination_folder, conflict_resolution_method, is_dry_run, rename_in_place):
    log_entries = []
    processed_files = 0
    conflicts_resolved = 0
    exif_failures = 0
    conflict_files = []
    temp_filenames = []  # To handle conflicts during dry_run

    for filename in os.listdir(source_folder):
        original_extension = os.path.splitext(filename)[1].lower()
        if not original_extension in ('.jpg', '.jpeg'):
            continue

        file_path = os.path.join(source_folder, filename)
        original_date = read_exif_data(file_path)
        if not original_date:
            exif_failures += 1
            log_entries.append([filename, '', 'TRUE', 'FALSE'])
            continue
        
        new_filename = get_new_filename(original_date, 0, conflict_resolution_method, original_extension)
        conflict_count = temp_filenames.count(new_filename) if is_dry_run else 0
        new_filename = get_new_filename(original_date, conflict_count, conflict_resolution_method, original_extension)
        
        if is_dry_run:
            temp_filenames.append(new_filename)
        else:
            new_file_path = os.path.join(destination_folder if not rename_in_place else source_folder, new_filename)
            if not rename_in_place:
                shutil.copy2(file_path, new_file_path)
            else:
                os.rename(file_path, new_file_path)
        
        if conflict_count > 0:
            conflicts_resolved += 1
            conflict_files.append(filename)

        log_entries.append([filename, new_filename, 'FALSE', 'TRUE' if conflict_count > 0 else 'FALSE'])
        processed_files += 1

    log_path = save_log_to_excel(log_entries, source_folder, conflict_resolution_method)

    print(f"Processed {processed_files} files, resolved {conflicts_resolved} conflicts, encountered {exif_failures} EXIF read failures.")
    if conflict_files:
        print("Conflicts occurred in the following files:")
        for file in conflict_files:
            print(f" {file}")
    print(f"Log saved to: {log_path}")

# Run the script
process_images_in_folder(source_folder, destination_folder, conflict_resolution_method, is_dry_run, rename_in_place)
